In [13]:
#Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from scipy.stats import iqr
from scipy.stats import linregress
import numpy as np
import pprint as pp
import gmaps
import requests
from key import g_key


In [3]:
# Read in the collision violations data set
collision = pd.read_csv('pd_collisions_datasd_v1.csv')
collision.head(5)

,report_id,date_time,police_beat,address_number_primary,address_pd_primary,address_road_primary,address_sfx_primary,address_pd_intersecting,address_name_intersecting,address_sfx_intersecting,violation_section,violation_type,charge_desc,injured,killed,hit_run_lvl
0,180282,2018-01-01 00:30:00,124,6500,,EL CAMINO DE TEATRO,,,,,22350,VC,VIOLATION OF BASIC SPEED LAW SPEED UNSAFE FOR ...,0,0,NaN
1,180310,2018-01-01 01:00:00,616,500,,ROSECRANS,STREET,,,,22350,VC,VIOLATION OF BASIC SPEED LAW SPEED UNSAFE FOR ...,0,0,MISDEMEANOR
2,180573,2018-01-01 01:26:00,721,0,,PALM,AVENUE,,18TH,STREET,21954A,VC,PEDESTRIANS OUTSIDE CROSSWALKS,0,1,NaN
3,180317,2018-01-01 02:05:00,512,3100,,LOGAN,AVENUE,,,,22450(A),VC,STOP REQUIREMENTS STATE STOP LINE,0,0,MISDEMEANOR
4,180319,2018-01-01 02:26:00,439,6200,,DIVISION,STREET,,,,22107,VC,TURNING MOVEMENTS AND REQUIRED SIGNALS,0,0,MISDEMEANOR


In [51]:
#Replace address "0" values with blank
collision['address_number_primary'] = collision['address_number_primary'].replace(0, '')

#Concatenate addresses together
collision["full_address"] = collision["address_number_primary"].astype(str) + " " 
                            + collision["address_pd_primary"] + " " 
                            + collision['address_road_primary'] + " " 
                            + collision['address_sfx_primary'] 
                            + " San Diego, California"
            
#Export new dataframe to csv
collision.to_csv("Collision_Data_PrePull.csv", index= False)
collision.head(5)

,report_id,date_time,police_beat,address_number_primary,address_pd_primary,address_road_primary,address_sfx_primary,address_pd_intersecting,address_name_intersecting,address_sfx_intersecting,violation_section,violation_type,charge_desc,injured,killed,hit_run_lvl,full_address
0,180282,2018-01-01 00:30:00,124,6500,,EL CAMINO DE TEATRO,,,,,22350,VC,VIOLATION OF BASIC SPEED LAW SPEED UNSAFE FOR ...,0,0,NaN,"6500 EL CAMINO DE TEATRO San Diego, Califo..."
1,180310,2018-01-01 01:00:00,616,500,,ROSECRANS,STREET,,,,22350,VC,VIOLATION OF BASIC SPEED LAW SPEED UNSAFE FOR ...,0,0,MISDEMEANOR,"500 ROSECRANS STREET San Diego, California"
2,180573,2018-01-01 01:26:00,721,,,PALM,AVENUE,,18TH,STREET,21954A,VC,PEDESTRIANS OUTSIDE CROSSWALKS,0,1,NaN,"PALM AVENUE San Diego, California"
3,180317,2018-01-01 02:05:00,512,3100,,LOGAN,AVENUE,,,,22450(A),VC,STOP REQUIREMENTS STATE STOP LINE,0,0,MISDEMEANOR,"3100 LOGAN AVENUE San Diego, California"
4,180319,2018-01-01 02:26:00,439,6200,,DIVISION,STREET,,,,22107,VC,TURNING MOVEMENTS AND REQUIRED SIGNALS,0,0,MISDEMEANOR,"6200 DIVISION STREET San Diego, California"


In [53]:
#Assign arrays for correct address, lat, lng data
correct_address = []
address_lat = []
address_lng = []

#Iterate through rows of address dataframe
for index, row in collision.iterrows():
    
    #Grab full address from column
    crash_address = row['full_address']

    #Define parameters for api pull url
    params = {
        "address": crash_address,
        "key": g_key
    }
    
    #Assign base url
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    #Pull request
    response = requests.get(base_url, params).json()
    
    #If response is "Zero Results" append formatted address, lat, lng to arrays
    if response['status'] != "ZERO_RESULTS":
    
        correct_address.append(response["results"][0]["formatted_address"])
        address_lat.append(response["results"][0]["geometry"]["location"]["lat"])
        address_lng.append(response["results"][0]["geometry"]["location"]["lng"])
    
    else:
        print(f"Unable to find address details for {crash_address}")
        
#Display formatted address, address lat, address lat in dataframe
full_address_df = pd.DataFrame({"Full_Address": correct_address,
                          "Address_Lat": address_lat,
                          "Address_Lng": address_lng})

#Export to csv
full_address_df.to_csv("Crash_Addresses_Coordinates.csv", index= False)

Unable to find address details for    5:40 PM   San Diego, California


In [9]:
#Read in csv
full_address_df = pd.read_csv('Crash_Addresses_Coordinates.csv')
full_address_df.head(5)

,Full_Address,Address_Lat,Address_Lng
0,"6500 El Camino Del Teatro, La Jolla, CA 92037,...",32.830196,-117.262811
1,"500 Rosecrans St, San Diego, CA 92106, USA",32.711431,-117.238088
2,"Palm Ave, San Diego, CA 92123, USA",32.802492,-117.140552
3,"3100 Logan Ave, San Diego, CA 92113, USA",32.696992,-117.127278
4,"6200 Division St, San Diego, CA 92114, USA",32.695031,-117.064637


In [10]:
#Split full addresses by comma
split_addresses_df = full_address_df["Full_Address"].str.split(",", n = 3, expand = True) 

#Split zip and "CA" into separate columns
split_zip = split_addresses_df[2].str.split(" ", n = 2, expand = True) 

#Rename column 2 Zip
full_address_df['Zip'] = split_zip[2]

#Get unique values for Zip
full_address_df['Zip'].unique()

#Filter out zip codes that are not in San Diego
sd_zip_only_df = full_address_df[(full_address_df['Zip'] != '67301') & 
                     (full_address_df['Zip'] != '60174') & 
                     (full_address_df['Zip'] != "Diego") &
                     (full_address_df['Zip'] != 'None') & 
                     (full_address_df['Zip'] != '95017') &
                     (full_address_df['Zip'] != '90015') &
                    (full_address_df['Zip'] != '92058')].dropna()
#Group by full address
grouped_by_address = sd_zip_only_df.groupby(['Full_Address'])

#Get count of each address
count_per_address = grouped_by_address["Full_Address"].count()

#Pull unique address lat, lngs for each address
address_lat = grouped_by_address["Address_Lat"].unique().str[0]
address_lng = grouped_by_address["Address_Lng"].unique().str[0]

#Create dataframe with count per address, and lat, lng associated with that address
count_per_address_df = pd.DataFrame({"Accident Count": count_per_address, "Address Lat": address_lat, "Address Lng": address_lng})

#Sort by descending on Accident Count column in dataframe
count_per_address_df.sort_values(by=['Accident Count'], ascending=False)

,Accident Count,Address Lat,Address Lng
Full_Address,,,
"7000 Friars Rd, San Diego, CA 92108, USA",69,32.769389,-117.170188
"5900 Linda Vista Rd, San Diego, CA 92110, USA",38,32.770330,-117.188301
"655 Sixth Ave #413, San Diego, CA 92101, USA",37,32.712074,-117.158949
"4100 University Ave, San Diego, CA 92105, USA",32,32.749720,-117.106939
"3100 Sports Arena Blvd, San Diego, CA 92110, USA",32,32.751934,-117.205324
...,...,...,...
"3400 Sunset Ln, San Diego, CA 92173, USA",1,32.558206,-117.059252
"3400 Sterne St, San Diego, CA 92106, USA",1,32.735219,-117.226193
"3400 State St, San Diego, CA 92103, USA",1,32.739068,-117.175323


In [12]:
# Configure gmaps
gmaps.configure(g_key)

# Assign map display to fig
fig = gmaps.figure()

# Store latitude and longitude in locations
locations = count_per_address_df[["Address Lat", "Address Lng"]]

# Assign crashes value to Accident Count column
crashes = count_per_address_df['Accident Count']

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=crashes, 
                                 dissipating=False, max_intensity=69,
                                 point_radius=.004)
#Specify plotting parameters
figure_layout = {
    'width': '400px',
    'height': '4000px',
    'border': '1px solid black',
    'padding': '1px'
}

#Assign layout parameters
gmaps.figure(layout=figure_layout)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))